In [ ]:
#initialize code and import packages
from run_multiscale_mapping_bgc import *
import dask
from dask.distributed import LocalCluster, Client, wait
import os
import convenient_bgc as cv
import xarray as xr
import numpy as np
import math

In [ ]:
##FOR USE ON SCRATCH
scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")

dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

client = Client(scheduler_file=scheduler_file)
client

In [ ]:
client.upload_file("njit_options_bgc.py")
client.upload_file("basisfunctions_bgc.py")
client.upload_file("covariancefunctions_bgc.py")
client.upload_file("distancefunctions_bgc.py")
client.upload_file("dotproduct_bgc.py")
client.upload_file("solvers_bgc.py")
client.upload_file("map_anom_bgc.py")
client.upload_file("batch_map_anom_bgc.py")
client.upload_file("distancefunctions_bgc.py")
client.upload_file("mean_field_lsq_bgc.py")
client.upload_file("negative_loglikelihood_bgc.py")
client.upload_file("maximum_likelihood_bgc.py")

In [ ]:
### USE FOR TESTING ON HOMES

cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
print('External dashboard-link: ', client.dashboard_link)
print(client)

In [ ]:
#window_size_betaparam = math.sqrt(1.0 * 500e3**2)
window_size_covparam = math.sqrt(2.0 * 700e3**2) # 2 x the area of a window w/ 700e3 meter radius
timewindow_size = 250 # Days
datafield = 'oxygen'

NHarm = 2
testn = 0 #test number

##do i even need these data files?
tsdatafile = '/global/homes/k/kefalc/code/data_files/Final/floats_years_2015_2019_temperature_salinity_oxygen_10daily_SO_30S_edit.nc' 
bgcdatafile = '/global/homes/k/kefalc/code/data_files/Final/floats_years_2015_2019_temperature_salinity_oxygen_10daily_SO_30Skeep_random_floats_15perc.nc'
#'/global/homes/k/kefalc/code/data_files/floats_years_2015_2019_temperature_salinity_oxygen_10daily_SO_Testkeep_random_floats_15perc_edit.nc'

maskfile = '/global/homes/k/kefalc/code/gpr-mapping-covar/test_mask.nc'

Nblocks = 1 
multivar_flag = 2 #1 for univariate, 2 for multivariate without noise, 3 for multivariate with noise
param_flag = 1 #0 for univariate, 1 for temperature, 2 for salinity
time_i = (np.datetime64('2015-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
time_f = (np.datetime64('2019-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0


grid_time_i = (np.datetime64('2017-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
grid_time_f = (np.datetime64('2017-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0

#outfile = '/global/homes/k/kefalc/code/'+'small_scale/'+str(np.int64(window_size_betaparam/1e3))+'_wscov'+str(np.int64(window_size_covparam/1e3))+'_plevel'+str(plevel)+'.nc'


In [ ]:
#%%time
for plevel in range(4,5):
    ##Input files       
    tsresidualfile = '/global/homes/k/kefalc/code/data_files/Final'+'/forbgc_residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc' ##this needs to be the file with only the locations that have BGC data
    #'/global/homes/k/kefalc/code/data_files/forbgc_residualfile_plevel3_dates16436.0_18261.0_test0.nc'
    
    
    bgcresidualfile = '/global/homes/k/kefalc/code/anom_files/final/oxy/res'+'/residualfile_plevel'+str(plevel)+'oxygen'+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc' 
    tscovparamfile = '/global/homes/k/kefalc/code/covar_files/final/TS/covparam/N5/covparam_ws989_N5_plevel'+str(plevel)+'_LBFGS_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    
    ##Files to be saved:
    bgccovparamfile =  '/global/homes/k/kefalc/code/covar_files/final'+'/' + datafield + '_covparam_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_LBFGS'+'_dates'+str(time_i)+'_'+str(time_f)+'_mvf'+str(multivar_flag)+'_param'+str(param_flag)+'_test'+str(testn)+'.nc'
    #'/global/homes/k/kefalc/code/covar_files/final/oxygen_covparam_ws989_N1_plevel3_LBFGS_dates16436.0_18261.0_mvf1_test0.nc'
    #'/global/homes/k/kefalc/code/covar_files/final'+'/' + datafield + '_covparam_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_LBFGS'+'_dates'+str(time_i)+'_'+str(time_f)+'_mvf'+str(multivar_flag)+'_test'+str(testn)+'.nc'
    BGC_gridanom = '/global/homes/k/kefalc/code/covar_files'+'/' + datafield + '_grid_anom_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_dates'+str(grid_time_i)+'_'+str(grid_time_f)+'_mvf'+str(multivar_flag)+'_param'+str(param_flag)+'_test'+str(testn)+'.nc'
    
    ##RUN CODE
    #bgc_covparam( plevel, datafield, bgcresidualfile, tsresidualfile, maskfile, tscovparamfile, bgccovparamfile, window_size_covparam, Nblocks, time_i, time_f, client, multivar_flag, param_flag)

    ##Grid anom
    if multivar_flag == 1: 
        grid_anom_seasonal_bgc( plevel, bgccovparamfile, bgcresidualfile, maskfile, window_size_covparam, timewindow_size, grid_time_i, grid_time_f, BGC_gridanom, client )

    else:
        tsonly_residualfile = ('/global/homes/k/kefalc/code/data_files/Final'+'/tsonly_forbgc_residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc')
        grid_anom_seasonal_multi( plevel, param_flag, bgccovparamfile, tscovparamfile, bgcresidualfile, tsresidualfile, tsonly_residualfile, maskfile, window_size_covparam, timewindow_size, grid_time_i, grid_time_f, BGC_gridanom, client)

In [ ]:
param_flag = 2

#%%time
for plevel in range(2,5):
    ##Input files       
    tsresidualfile = '/global/homes/k/kefalc/code/data_files/Final'+'/forbgc_residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc' ##this needs to be the file with only the locations that have BGC data
    #'/global/homes/k/kefalc/code/data_files/forbgc_residualfile_plevel3_dates16436.0_18261.0_test0.nc'
    
    
    bgcresidualfile = '/global/homes/k/kefalc/code/anom_files/final/oxy/res'+'/residualfile_plevel'+str(plevel)+'oxygen'+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc' 
    tscovparamfile = '/global/homes/k/kefalc/code/covar_files/final/TS/covparam/N5/covparam_ws989_N5_plevel'+str(plevel)+'_LBFGS_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    
    ##Files to be saved:
    bgccovparamfile =  '/global/homes/k/kefalc/code/covar_files/final'+'/' + datafield + '_covparam_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_LBFGS'+'_dates'+str(time_i)+'_'+str(time_f)+'_mvf'+str(multivar_flag)+'_param'+str(param_flag)+'_test'+str(testn)+'.nc'
    #'/global/homes/k/kefalc/code/covar_files/final/oxygen_covparam_ws989_N1_plevel3_LBFGS_dates16436.0_18261.0_mvf1_test0.nc'
    #'/global/homes/k/kefalc/code/covar_files/final'+'/' + datafield + '_covparam_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_LBFGS'+'_dates'+str(time_i)+'_'+str(time_f)+'_mvf'+str(multivar_flag)+'_test'+str(testn)+'.nc'
    BGC_gridanom = '/global/homes/k/kefalc/code/covar_files'+'/' + datafield + '_grid_anom_ws'+str(np.int64(window_size_covparam/1e3))+'_N'+str(Nblocks)+'_plevel'+str(plevel)+'_dates'+str(grid_time_i)+'_'+str(grid_time_f)+'_mvf'+str(multivar_flag)+'_param'+str(param_flag)+'_test'+str(testn)+'.nc'
    
    ##RUN CODE
    bgc_covparam( plevel, datafield, bgcresidualfile, tsresidualfile, maskfile, tscovparamfile, bgccovparamfile, window_size_covparam, Nblocks, time_i, time_f, client, multivar_flag, param_flag)

In [ ]:
client.shutdown()